In [4]:
import asyncio
import websockets
import json
import pandas as pd
apiKeys=pd.read_csv("/Users/maier-borst.h/Documents/apiKeys.csv", sep=";")
apiKeyAIS=apiKeys[apiKeys["Service"]=="AISStream"]["key"].iloc[0]
apiKeyAIS
from datetime import datetime, timezone

async def connect_ais_stream():

    async with websockets.connect("wss://stream.aisstream.io/v0/stream") as websocket:
        subscribe_message = {"APIKey": apiKeyAIS,  # Required !
                             "BoundingBoxes": [[[-120, -10], [0, 10]]], # Required!
                             "FilterMessageTypes": ["PositionReport"]} # Optional!

        subscribe_message_json = json.dumps(subscribe_message)
        await websocket.send(subscribe_message_json)
        print(websocket)
        async for message_json in websocket:
            message = json.loads(message_json)
            message_type = message["MessageType"]
            websocket.close();

            if message_type == "PositionReport":
                # the message parameter contains a key of the message type which contains the message itself
                ais_message = message['Message']['PositionReport']
                print(f"[{datetime.now(timezone.utc)}] ShipId: {ais_message['UserID']} Latitude: {ais_message['Latitude']} Latitude: {ais_message['Longitude']}")



In [23]:
await(connect_ais_stream())

In [25]:
async def connect_ais_stream():
    async with websockets.connect("wss://stream.aisstream.io/v0/stream") as websocket:
        subscribe_message = {
            "APIKey": apiKeyAIS,  # Required !
            "BoundingBoxes": [[[-120, -10], [0, 10]]],  # Required!
            "FilterMessageTypes": ["PositionReport"]  # Optional!
        }

        subscribe_message_json = json.dumps(subscribe_message)
        await websocket.send(subscribe_message_json)

        try:
            await asyncio.wait_for(process_messages(websocket), timeout=10)
        except asyncio.TimeoutError:
            print("Timeout occurred. Stopping execution.")

async def process_messages(websocket):
    async for message_json in websocket:
        message = json.loads(message_json)
        message_type = message["MessageType"]
        websocket.close()

        if message_type == "PositionReport":
            ais_message = message['Message']['PositionReport']
            print(f"[{datetime.now(timezone.utc)}] ShipId: {ais_message['UserID']} Latitude: {ais_message['Latitude']} Latitude: {ais_message['Longitude']}")

# Call the function


RuntimeError: asyncio.run() cannot be called from a running event loop

In [26]:
(connect_ais_stream())

<coroutine object connect_ais_stream at 0x119282040>